In [1]:
import pandas as pd
import numpy as np
import sys
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
def MD(x):
    n=len(x)
    y=np.array(x)
    mean=np.mean(y, axis=0)
#     print("mean of each dimension")
#     print(mean)
#     print("")
    for x in y:
        z=y-mean
    #print("deviation matrix")
    #print(z)
    k=z.T
    #print(k.shape)
    t=np.matmul(k, z)/(n-1)
#     print("co-variance matrix")
#     print(t)
#     print("inverse covariance matrix")
    p = np.linalg.inv(t)
#     print(p)
    return p,mean, np.linalg.det(t) 

In [3]:
def medv(covar,mean,v):
    vect=np.array(v)
    vect=vect-mean;
    
    vectt=np.array(vect.T)
   
    z=np.matmul(vectt,covar)
    return np.sqrt(np.matmul(z,vect)*0.5)

In [4]:
def predict(incovar,mean_c,y,pwi,det):
    dis=[]

    for i  in range(3):
        dis.append(medv(incovar[i],mean_c[i],y))
    res=[]
    res=np.log(pwi)-dis-np.log(det)
    res=list(res)
    return res.index(max(res))
    

In [5]:
def NB(data):
    data.head()
    data['species']=data["species"].astype("category")
    data['species']=data["species"].cat.codes
    X=data
    y=data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    df=[]
    for i in range(3):
        df.append((X_train[X_train["species"]==i]))
#     df[1]=(X_train[X_train["species"]==1])
#     df[2]=(X_train[X_train["species"]==2])
    df[0]=df[0].iloc[:,:-1]
    df[1]=df[1].iloc[:,:-1]
    df[2]=df[2].iloc[:,:-1]
    pwi=[]
    for i  in range(3):
        pwi.append(len(df[i])/(len(df[0])+len(df[1])+len(df[2])))
    X_test=X_test.iloc[:,:-1]
    X_test=X_test.values
    incovar=[]
    mean_c=[]
    det=[]
    for i in range(3):
        covar,mean,det_i=MD(df[i])
        incovar.append(covar)
        mean_c.append(mean)
        det.append(det_i)
    y_pred=[]
    for i in range(3):
        print("invcovariance of class  " +str(i))
        print(incovar[i])
        print("mean of class "+str(mean[i]))
        
        print("determinant of covariance matrix of class " +str(det[i]))
    
    for i in range(len(X_test)):
           y_pred.append(predict(incovar,mean_c,X_test[i],pwi,det))
    return y_pred,y_test

In [6]:
data=pd.read_csv("IRIS.csv")
target_names=data["species"].unique()

In [11]:
y_pred,y_test=NB(data)

invcovariance of class  0
[[ 17.9672428  -11.99606559  -6.31379103  -2.04050942]
 [-11.99606559  15.7190156   -0.1571023   -3.52989501]
 [ -6.31379103  -0.1571023   60.77171554 -24.73554188]
 [ -2.04050942  -3.52989501 -24.73554188 101.5594296 ]]
mean of class 6.638636363636365
determinant of covariance matrix of class 1.5324203799090606e-06
invcovariance of class  1
[[  9.95448984  -3.94038886  -9.68031051   8.76907022]
 [ -3.94038886  16.38693257   2.6877528  -15.55514891]
 [ -9.68031051   2.6877528   22.02656959 -31.8070975 ]
 [  8.76907022 -15.55514891 -31.8070975   88.2496317 ]]
mean of class 2.9886363636363638
determinant of covariance matrix of class 2.266150525091067e-05
invcovariance of class  2
[[ 12.30668944  -3.48765214 -11.86806516   1.12520534]
 [ -3.48765214  14.37944062   1.56758672  -8.01565586]
 [-11.86806516   1.56758672  15.595557    -3.30390592]
 [  1.12520534  -8.01565586  -3.30390592  22.39509436]]
mean of class 5.5659090909090905
determinant of covariance matrix

In [12]:
accuracy_score(y_pred,y_test)

0.9666666666666667

In [13]:
print(confusion_matrix(y_test, y_pred))

[[11  0  0]
 [ 0 13  0]
 [ 0  1  5]]


In [14]:
from sklearn.metrics import classification_report

In [15]:
print(classification_report(y_test,y_pred,target_names=target_names))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       0.93      1.00      0.96        13
 Iris-virginica       1.00      0.83      0.91         6

      micro avg       0.97      0.97      0.97        30
      macro avg       0.98      0.94      0.96        30
   weighted avg       0.97      0.97      0.97        30



In [21]:

cm = confusion_matrix(y_test, y_pred)
#Now the normalize the diagonal entries
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [22]:
class_acc=cm.diagonal()
class_acc=list(class_acc)

In [23]:
class_acc

[1.0, 1.0, 0.8333333333333334]

In [7]:
print(data.groupby('species'))

In [9]:
x=data.groupby('species')